In [1]:
from fh import FH 
import fh
from treeBuilder import TreeBuilder as TB
import PydalChanel as pydal
import copy
import pickle
import arpeggiator

lp = FH()


oneHitShift = fh.oneHitShift

read = pydal.read
end = pydal.end
tempo = pydal.tempo
rootScale = lp.rootScale
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)

idv = lambda a : a
codec = lambda buf: fh.noteListToHitList(fh.flattenByBeat(fh.notesByBeat(fh.hitListToNoteList(buf))))

lp.addForwardingHandler(0, arpeggiator.Arpeggiator(0, pydal.pydalInstance, read("0 1 2", 1, "arp")))

In [ ]:
lp.buttonForwardingHandlers

In [ ]:
a = read("{bd (lt*2, lt, lt), hc <~, sn hc> ~ hc:2}")
b = read("{(bd ~, sn:3 <~, cp>)}")
c = read("((bd:2*2, [bd:3 bd:3] bd), [sn:5 [bd sn:3]], [[bd sn:3] bd], <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>)")
c2 = read("(bd:2*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>", 4)
d = read("{~ ~, bin ~ bin ~}")
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")


demo = read("bd [hc:2 hc]")

f = read("aa:4_4.75*2 aa:5_5.75*3", 2, 'sample')
f2 = read("hc*2 cp*3", 2)
d2 = read("{~ ~ , bd ~ sn:3 ~}", 4)
g = read("{~ ~ , ab:0_2 ~ ab:3_5 ~}", 4, 'sample')

In [ ]:
ch1.stop()

In [ ]:
lp.saveScenesToFile("song2_good.scenes")

In [ ]:
lp.loadScenesFromFile("postPedalTest1.scenes")


In [ ]:
lp.loadScenesFromFile("song1e.scenes")

In [ ]:
rootScale(1, lp.roots[1], "0,1,2,3,4,5,6,7,8,9,10,11")

In [ ]:
lp.saveScenesToFile("song1slow.scenes")

In [ ]:
st = '69 0.49950693402567,80,101,1,on-0.12300503053356,80,0,1,off-0.37064133602195,75,98,1,on-0.10712906254986,75,0,1,off-0.36297831440561,75,103,1,on-0.090360763184094,75,0,1,off-0.40264527718247,75,108,1,on-0.098848608124257,75,0,1,off-0.39581608216994,75,103,1,on-0.074119008398242,75,0,1,off-0.41079760461554,75,111,1,on-0.073419676461185,75,0,1,off-0.41209621156585,75,112,1,on-0.12404576558649,75,0,1,off-0.45459032517527,0,0,0,timeAfterLastHit 216'
for s in st.split('-'):
    print s

In [ ]:
lp.startChannel(1, lambda buf: fh.randTranspose(buf, lp.roots[0], lp.scales[0]), lp.loops[0][0])

In [19]:
lp.stopChannel(2)

In [9]:
lp.startChannel(1, lambda buf: fh.warp(buf, 3, 1), lp.loops[1][3])

In [18]:
lp.startChannel(2, lambda buf: fh.warp(buf, 3, .2, 1.3), lp.loops[3][0])